## Patching Experiments

### Load the dataset

In [1]:
import json
with open('l1.json', 'r') as f:
    l1_dict=json.load(f)
l1=list(l1_dict.keys())
neg_adj=list(l1_dict.values())
with open('l2.json', 'r') as f:
    l2_dict=json.load(f)
l2=list(l2_dict.keys())
with open('l3.json', 'r') as f:
    l3_dict=json.load(f)
l3=list(l3_dict.keys())
pos_adj=list(l3_dict.values())
with open('l4.json', 'r') as f:
    l4_dict=json.load(f)
l4=list(l4_dict.keys())
with open('l5.json', 'r') as f:
    l5_dict=json.load(f)
l5=list(l5_dict.keys())


### Setting everything up for Trasnformer lens library

In [2]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio
DEBUG_MODE = False
IN_COLAB = False
print("Running as a Jupyter notebook - intended for development only!")
from IPython import get_ipython
ipython = get_ipython()
# Code to automatically update the HookedTransformer code as its edited without restarting the kernel
ipython.magic("load_ext autoreload")
ipython.magic("autoreload 2")

pio.renderers.default = "vscode"
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader
from jaxtyping import Float, Int

from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML
import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache
torch.set_grad_enabled(False)
from neel_plotly import line, imshow, scatter
import transformer_lens.patching as patching
model = HookedTransformer.from_pretrained("gpt2-small")
prompts = ['The day is bright but the night is']
not_id=model.to_single_token(" not")
bright_id=model.to_single_token(" bright")
clean_tokens = model.to_tokens(prompts)
print("Clean string 0", model.to_string(clean_tokens[0]))
logits,cache=model.run_with_cache(clean_tokens)
id=logits[...,-1,:].argmax()
model.to_string(id)

import plotly.io as pio
pio.renderers.default = "vscode"


def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def line(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.line(utils.to_numpy(tensor), labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)


def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

Running as a Jupyter notebook - intended for development only!


/tmp/ipykernel_32938/1326878066.py:9: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("load_ext autoreload")
/tmp/ipykernel_32938/1326878066.py:10: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("autoreload 2")
Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-small into HookedTransformer
Clean string 0 <|endoftext|>The day is bright but the night is


# Activation Patching

In [10]:
import transformer_lens.patching as patching
clean_tokens=model.to_tokens(l1)
clean_tokens.shape
corrupted_tokens=model.to_tokens(l5)
answers=[(i,j) for i,j in zip(neg_adj,pos_adj)]
answer_token_indices = torch.tensor([[model.to_single_token(" "+answers[i][j]) for j in range(2)] for i in range(len(answers))], device=model.cfg.device)
l5=[i.replace("but","and") for i in l1]


def get_logit_diff(logits, answer_token_indices=answer_token_indices):
    if len(logits.shape)==3:
        # Get final logits only
        logits = logits[:, -2, :]
    correct_logits = logits.gather(1, answer_token_indices[:, 0].unsqueeze(1))
    incorrect_logits = logits.gather(1, answer_token_indices[:, 1].unsqueeze(1))
    return (correct_logits - incorrect_logits).mean()
#The l1 dataset has te answer positive adjective
#The l5 dataset has te answer positive adjective
clean_logits, clean_cache = model.run_with_cache(clean_tokens)
corrupted_logits, corrupted_cache = model.run_with_cache(corrupted_tokens)
clean_logit_diff = get_logit_diff(clean_logits, answer_token_indices).item()
corrupted_logit_diff = get_logit_diff(corrupted_logits, answer_token_indices).item()

In [9]:
l5=[i.replace("but","and") for i in l1]


['The day is bright and the night is',
 'The summer is hot and the winter is',
 'The victory is sweet and the defeat is',
 'The fire is warm and the watter is',
 'The war is bad and the peace is',
 'The man is strong and the woman is',
 'The king is good and the queen is']

In [11]:
clean_tokens=model.to_tokens(l1)
clean_logits, clean_cache = model.run_with_cache(clean_tokens)
clean_logits[:, -2, :].argmax(dim=-1)


tensor([ 3223,  4692, 12922,   318,   922,  4939,  2089])

In [12]:
CLEAN_BASELINE = clean_logit_diff
CORRUPTED_BASELINE = corrupted_logit_diff
def sb_metric(logits, answer_token_indices=answer_token_indices):
    return (get_logit_diff(logits, answer_token_indices) - CORRUPTED_BASELINE) / (CLEAN_BASELINE  - CORRUPTED_BASELINE)

resid_act_mlp_out_results = patching.get_act_patch_mlp_out(model, corrupted_tokens, clean_cache, sb_metric)

  0%|          | 0/108 [00:00<?, ?it/s]

In [13]:
imshow(resid_act_mlp_out_results[:,1:9], 
       yaxis="Layer", 
       xaxis="Position", 
       x=[f"{tok} {i}" for i, tok in enumerate(model.to_str_tokens(clean_tokens[0][1:9]))],
       title="resid_act_mlp_out_results Activation Patching")

In [24]:
ALL_HEAD_LABELS = [f"L{i}H{j}" for i in range(model.cfg.n_layers) for j in range(model.cfg.n_heads)]
if True:
    attn_head_out_act_patch_results = patching.get_act_patch_attn_head_out_by_pos(model, corrupted_tokens, clean_cache, sb_metric)
    attn_head_out_act_patch_results = einops.rearrange(attn_head_out_act_patch_results, "layer pos head -> (layer head) pos")


  0%|          | 0/1296 [00:00<?, ?it/s]

In [28]:
imshow(attn_head_out_act_patch_results[:,1:9], 
       yaxis="Layer", 
       xaxis="Position", 
       x=[f"{tok} {i}" for i, tok in enumerate(model.to_str_tokens(clean_tokens[0][1:9]))],
       title="attn_head_out_act_patch_results Activation Patching")

In [40]:
#["Residual Stream", "Attn Output", "MLP Output"]

every_block_result = patching.get_act_patch_block_every(model, corrupted_tokens, clean_cache, sb_metric)


  0%|          | 0/108 [00:00<?, ?it/s]

  0%|          | 0/108 [00:00<?, ?it/s]

  0%|          | 0/108 [00:00<?, ?it/s]

In [41]:
imshow(every_block_result, facet_col=0, title="Activation Patching Per Block", xaxis="Position", yaxis="Layer", zmax=1, zmin=-1, x= [f"{tok}_{i}" for i, tok in enumerate(model.to_str_tokens(clean_tokens[0][0:9]))])

# Attribution patching

In [44]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from torchtyping import TensorType as TT
from typing import List, Union, Optional, Callable
from functools import partial
import copy
import itertools
import json

from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML, Markdown

import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

from neel_plotly import line, imshow, scatter
import transformer_lens.patching as patching

In [45]:
Metric = Callable[[TT["batch_and_pos_dims", "d_model"]], float]
model = HookedTransformer.from_pretrained("gpt2-small")
model.set_use_attn_result(True)

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-small into HookedTransformer


In [67]:
model.to_tokens(l1)

tensor([[50256,   464,  1110,   318,  6016,   475,   262,  1755,   318, 50256],
        [50256,   464,  3931,   318,  3024,   475,   262,  7374,   318, 50256],
        [50256,   464,  5373,   318,  6029,   475,   262,  7433,   318, 50256],
        [50256,   464,  2046,   318,  5814,   475,   262,   266,  1436,   318],
        [50256,   464,  1175,   318,  2089,   475,   262,  4167,   318, 50256],
        [50256,   464,   582,   318,  1913,   475,   262,  2415,   318, 50256],
        [50256,   464,  5822,   318,   922,   475,   262, 16599,   318, 50256]])

In [68]:
filter_not_qkv_input = lambda name: "_input" not in name
def get_cache_fwd_and_bwd(model, tokens, metric):
    model.reset_hooks()
    cache = {}
    def forward_cache_hook(act, hook):
        cache[hook.name] = act.detach()
    model.add_hook(filter_not_qkv_input, forward_cache_hook, "fwd")

    grad_cache = {}
    def backward_cache_hook(act, hook):
        grad_cache[hook.name] = act.detach()
    model.add_hook(filter_not_qkv_input, backward_cache_hook, "bwd")

    value = metric(model(tokens),)
    value.backward()
    model.reset_hooks()
    return value.item(), ActivationCache(cache, model), ActivationCache(grad_cache, model)

clean_value, clean_cache, clean_grad_cache = get_cache_fwd_and_bwd(model, clean_tokens[0], sb_metric)
print("Clean Value:", clean_value)
print("Clean Activations Cached:", len(clean_cache))
print("Clean Gradients Cached:", len(clean_grad_cache))
corrupted_value, corrupted_cache, corrupted_grad_cache = get_cache_fwd_and_bwd(model, corrupted_tokens, sb_metric)
print("Corrupted Value:", corrupted_value)
print("Corrupted Activations Cached:", len(corrupted_cache))
print("Corrupted Gradients Cached:", len(corrupted_grad_cache))

RuntimeError: Size does not match at dimension 0 expected index [7, 1] to be smaller than self [1, 50257] apart from dimension 1

In [72]:
answer_token_indices

tensor([[ 3223,  6016],
        [ 4692,  3024],
        [12922,  6029],
        [ 4692,  5814],
        [  922,  2089],
        [ 4939,  1913],
        [ 2089,   922]])

In [ ]:
def get_head_vector_grad_input_from_grad_cache(
        grad_cache: ActivationCache, 
        activation_name: Literal["q", "k", "v"],
        layer: int
    ) -> TT["batch", "pos", "head_index", "d_model"]:
    vector_grad = grad_cache[activation_name, layer]
    ln_scales = grad_cache["scale", layer, "ln1"]
    attn_layer_object = model.blocks[layer].attn
    if activation_name == "q":
        W = attn_layer_object.W_Q
    elif activation_name == "k":
        W = attn_layer_object.W_K
    elif activation_name == "v":
        W = attn_layer_object.W_V
    else:
        raise ValueError("Invalid activation name")

    return einsum("batch pos head_index d_head, batch pos, head_index d_model d_head -> batch pos head_index d_model", vector_grad, ln_scales.squeeze(-1), W)

def get_stacked_head_vector_grad_input(grad_cache, activation_name: Literal["q", "k", "v"]) -> TT["layer", "batch", "pos", "head_index", "d_model"]:
    return torch.stack([get_head_vector_grad_input_from_grad_cache(grad_cache, activation_name, l) for l in range(model.cfg.n_layers)], dim=0)

def get_full_vector_grad_input(grad_cache) -> TT["qkv", "layer", "batch", "pos", "head_index", "d_model"]:
    return torch.stack([get_stacked_head_vector_grad_input(grad_cache, activation_name) for activation_name in ['q', 'k', 'v']], dim=0)

def attr_patch_head_path(
        clean_cache: ActivationCache, 
        corrupted_cache: ActivationCache, 
        corrupted_grad_cache: ActivationCache
    ) -> TT["qkv", "dest_component", "src_component", "pos"]:
    """
    Computes the attribution patch along the path between each pair of heads.

    Sets this to zero for the path from any late head to any early head

    """
    start_labels = HEAD_NAMES
    end_labels = HEAD_NAMES_QKV
    full_vector_grad_input = get_full_vector_grad_input(corrupted_grad_cache)
    clean_head_result_stack = clean_cache.stack_head_results(-1)
    corrupted_head_result_stack = corrupted_cache.stack_head_results(-1)
    diff_head_result = einops.rearrange(
        clean_head_result_stack - corrupted_head_result_stack,
        "(layer head_index) batch pos d_model -> layer batch pos head_index d_model",
        layer = model.cfg.n_layers,
        head_index = model.cfg.n_heads,
    )
    path_attr = einsum(
        "qkv layer_end batch pos head_end d_model, layer_start batch pos head_start d_model -> qkv layer_end head_end layer_start head_start pos", 
        full_vector_grad_input, 
        diff_head_result)
    correct_layer_order_mask = (
        torch.arange(model.cfg.n_layers)[None, :, None, None, None, None] > 
        torch.arange(model.cfg.n_layers)[None, None, None, :, None, None]).to(path_attr.device)
    zero = torch.zeros(1, device=path_attr.device)
    path_attr = torch.where(correct_layer_order_mask, path_attr, zero)

    path_attr = einops.rearrange(
        path_attr,
        "qkv layer_end head_end layer_start head_start pos -> (layer_end head_end qkv) (layer_start head_start) pos",
    )
    return path_attr, end_labels, start_labels

head_path_attr, end_labels, start_labels = attr_patch_head_path(clean_cache, corrupted_cache, corrupted_grad_cache)